In [1]:
import pandas as pd
import seaborn as sns
from scripts.data_processing import process_data
import os
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../data/clean/processed_met_data.csv')

In [24]:
def met_season(month):    
    if month in [12, 1, 2]:
        return 'summer'
    elif month in [3, 4, 5]:
        return 'fall'
    elif month in [6, 7, 8]:
        return 'winter'
    elif month in [9, 10, 11]:
        return 'spring'

In [25]:
df['date'] = pd.to_datetime(df['date'])

In [26]:
df['season'] = df.apply(lambda x: met_season(x['date'].month), axis = 1)

df['season'] = df.apply(lambda x: met_season(str(x['date'].day) + '/' + str(x['date'].month)), axis = 1)

In [27]:
def calculate_perceived_temperature(celsius, humidity, wind_speed):
    if celsius >= 27:
        f = celsius * 1.8 + 32
        return -42.379 + (2.04901523*f) + (10.14333127*humidity) - (.22475541*f*humidity) - (.00683783*f*f) - (.05481717*humidity*humidity) + (.00122874*f*f*humidity) + (.00085282*f*humidity*humidity) - (.00000199*f*f*humidity*humidity)
    elif celsius <= 10:
        return (12.1452 + 11.6222 * np.sqrt(wind_speed) - 1.16222 * wind_speed) * (33 - celsius)
    else:
        return celsius

In [28]:
df['perceived_temperature'] = df.apply(lambda x: 
                                       calculate_perceived_temperature(
                                           x['air_temp_dry_bulb'], 
                                           x['relative_humidity'], 
                                           x['wind_speed']), 
                                       axis = 1)

In [29]:
df['lag_1h'] = df['energy_demand'].shift(1, fill_value = 0)
df['lag_2h'] = df['energy_demand'].shift(2, fill_value = 0)
df['lag_3h'] = df['energy_demand'].shift(3, fill_value = 0)
df['lag_6h'] = df['energy_demand'].shift(6, fill_value = 0)
df['lag_12h'] = df['energy_demand'].shift(12, fill_value = 0)
df['lag_24h'] = df['energy_demand'].shift(24, fill_value = 0)
df['lag_48h'] = df['energy_demand'].shift(48, fill_value = 0)

In [30]:
df_dummied = pd.get_dummies(df.copy(), columns = ['season'])

In [32]:
df_dummied

,energy_demand,date,atmospheric_pressure_sea_level,global_radiation,air_temp_dry_bulb,dew_point_temp,relative_humidity,wind_speed,is_weekend_False,is_weekend_True,...,lag_2h,lag_3h,lag_6h,lag_12h,lag_24h,lag_48h,season_fall,season_spring,season_summer,season_winter
0,32183.885,2023-01-01 00:00:00,1016.228574,67.846630,22.115419,19.032145,83.665394,1.588810,False,True,...,0.000,0.000,0.000,0.000,0.000,0.000,False,False,True,False
1,32196.682,2023-01-01 01:00:00,1016.627640,44.927211,21.721706,19.064871,85.663689,1.518970,False,True,...,0.000,0.000,0.000,0.000,0.000,0.000,False,False,True,False
2,31539.570,2023-01-01 02:00:00,1016.550538,32.603317,21.480185,18.983210,86.285438,1.597481,False,True,...,32183.885,0.000,0.000,0.000,0.000,0.000,False,False,True,False
3,30699.367,2023-01-01 03:00:00,1016.096920,26.303430,21.142992,18.980286,87.728168,1.589240,False,True,...,32196.682,32183.885,0.000,0.000,0.000,0.000,False,False,True,False
4,30126.195,2023-01-01 04:00:00,1015.465413,26.258225,20.864910,18.885765,88.645976,1.472117,False,True,...,31539.570,32196.682,0.000,0.000,0.000,0.000,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17539,47703.680,2024-12-31 19:00:00,1011.644722,1296.609406,26.062858,19.124788,67.481601,2.074146,True,False,...,41469.657,40232.966,39397.240,35839.548,46413.934,44373.991,False,False,True,False
17540,46685.056,2024-12-31 20:00:00,1011.575858,869.627541,25.514393,19.235745,70.048826,1.862968,True,False,...,43726.802,41469.657,39283.626,36593.001,46533.297,44245.640,False,False,True,False
17541,42634.745,2024-12-31 21:00:00,1012.108050,419.033705,24.408913,19.172731,74.048826,1.692420,True,False,...,47703.680,43726.802,39548.759,37182.028,45683.012,43333.425,False,False,True,False
17542,39017.488,2024-12-31 22:00:00,1012.810692,155.109408,23.211686,19.260132,79.481967,1.574472,True,False,...,46685.056,47703.680,40232.966,38273.632,44069.754,42030.288,False,False,True,False
